In [1]:
import mtt
from os.path import join, realpath
from os import getcwd
wiring = mtt.Wiring.bare(['Block 1', 'Block 2', 'Block 3'])
wiring.blocks[0].Ctot.connect(wiring.blocks[1].Btot)
wiring.blocks[1].Ctot.connect(wiring.blocks[2].Btot)
wiring.blocks[2].Ctot.connect(wiring.blocks[2].Cfree)
wiring.blocks[0].Ctot.connect(wiring.blocks[0].Cfree)

wiring.blocks[2].rv_up.connect(wiring.blocks[1].Cdeg)
wiring.blocks[1].rv_up.connect(wiring.blocks[0].Cdeg)

wiring

In [2]:
model = mtt.MTT(wiring)
model.draw(3)

# Parameter config

In [3]:
wiring.blocks[0].Atot = 50.
wiring.blocks[0].Btot = 10.
wiring.blocks[0].KDfw = 50.
wiring.blocks[0].kr = 10.
wiring.blocks[0].kdeg = 1.
wiring.blocks[0].Dfree = 1.
wiring.blocks[0].KDrv = 1000.
wiring.blocks[0].ratC = 10.
wiring.blocks[0].n = 1
wiring.blocks[0].A_FB_EN = False
wiring.blocks[0].B_FB_EN = False

wiring.blocks[1].Atot = 50.
wiring.blocks[1].Btot = 0.
wiring.blocks[1].KDfw = 50.
wiring.blocks[1].kr = 1.
wiring.blocks[1].kdeg = 1.
wiring.blocks[1].Dfree = 1.
wiring.blocks[1].KDrv = 1000.
wiring.blocks[1].ratC = 0.
wiring.blocks[1].n = 1
wiring.blocks[1].A_FB_EN = False
wiring.blocks[1].B_FB_EN = True

wiring.blocks[2].Atot = 50.
wiring.blocks[2].Btot = 0.
wiring.blocks[2].KDfw = 50.
wiring.blocks[2].kr = 1.
wiring.blocks[2].kdeg = 1.
wiring.blocks[2].Dfree = 1.
wiring.blocks[2].KDrv = 1000.
wiring.blocks[2].ratC = 10.
wiring.blocks[2].n = 1
wiring.blocks[2].A_FB_EN = False
wiring.blocks[2].B_FB_EN = True

# Digitized current config

In [4]:
for block in wiring.blocks:
    print(block.getDigitizedParameterString(model))

# Generate SRAM Program

In [5]:
from mtt import addSRAMRule

block_indices = [15, 4, 10]
# block_indices = [2, 17, 8]
# block_indices = [12, 9, 11]
# block_indices = [5, 0, 16]

program = model.compileSRAMWithBlockRemap(block_indices)
rules = program.getRoutingRulesBinaryMatrix()

# ADC connections
# group, block, variable, wire
# variables:
Ctot_var  = 11
Bfree_var = 14
g = [int(blk/5) for blk in block_indices]
b = [blk%5 for blk in block_indices]
rules = addSRAMRule(rules,g[1],b[1],Bfree_var,88) # "A"
rules = addSRAMRule(rules,g[2],b[2],Bfree_var,89) # "B"
rules = addSRAMRule(rules,g[2],b[2],Ctot_var,90)  # "C"

# wire to ADC
for i in range(76,99+1):
    rules = addSRAMRule(rules,0,0,31,i)

In [6]:
import pprint
pp = pprint.PrettyPrinter(indent=2, depth=1, width=10)
pp.pprint(program.dump())

# Routing Configuration

One row per wire.

In [7]:
from numpy import array
binrules = array([bin(rule[0])[2:].zfill(10) for rule in rules])
print(binrules.reshape((-1,1)))

# SRAM Layout

Rows are binary routing config rules, cols are wires.

In [8]:
from scipy.io import savemat
sram_matrix = program.convertToSRAMMatrix(rules)
savemat('sram.mat', {'in_all': sram_matrix})

# Shift Registers

In [16]:
sr = model.compileShiftReg()
prog_vect = sr.getProgrammingVectorIndexed(block_indices=block_indices)
for k in range(4):
    prog_vect[280:287,k] = sr.toBinaryCurrent(*model.digitize(10.)) # Ione
for k in [0,2,3]:
    prog_vect[280:287,k] = array([True, False, True, False, False, False, True])
# import numpy
# numpy.set_printoptions(threshold=numpy.nan)
# print(prog_vect)
savemat('sr.mat', {'prog_vect': prog_vect})

# Simulink Export

In [10]:
model.exportToSimulink(join(realpath(getcwd()),'simulink'), 'cascade', 3)

# Simulation

In [11]:
# hardware simulation
model.reset()
for k in range(3):
    print('block {}: rate_fw = {}'.format(k,model.blocks[k].rate_fw))
    print('block {}: rate = {}'.format(k,model.blocks[k].rate))
r = model.simulate(0,10,100)
# print(r)
model.plot()

# SBML Simulation

In [12]:
# SBML comparison
import tellurium as te
with open('/home/poltergeist/devel/src/mtt/test-cases/fig-3-5a-cascade-reversible.sb') as f:
    rr_model = te.loada(f.read())
print(rr_model.getReactionRates())
rr_model.simulate(0,10,1000)
rr_model.plot()